In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import glob
import os
from os import walk
from os import path
from bs4 import BeautifulSoup
import requests
from pandas.api.types import CategoricalDtype
import varname
import math
from pandas import Index

os.chdir('C:\Świetlik\KU Leuven\MT Masters thesis\Data')
mypath =os.getcwd()

# pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

# Functions

In [2]:
def show_wide(output):
    option_value = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', 200)
    try:
        display(output)
    finally:
        pd.set_option('display.max_colwidth', option_value)

# Kategorie - all years

In [20]:
# 2004 special treatment
k04a = pd.read_csv('BBGD/2004/B4X_2004.csv')
k04a.drop('g1', axis=1, inplace=True)
k04a.rok_p = 2004

k04b =  pd.read_csv('BBGD/2004/br2004_01a_1.csv')
k04b = k04b[['nrg','klm', 'trb','woj']]

k04 = k04a.merge(k04b, left_on='nrg', right_on='nrg')

keys = 'rok_p', 'nrg', 'waga', 'los', 'dochg', 'wydg', 'dgr', 'wtukg', 'wp', 'trb',
values = 'rok', 'nrg', 'mn', 'los', 'doch', 'wyd', 'dgr', 'wtuk', 'wp', 'typr',
rename_04_col = dict(zip(keys, values))

k04.rename(columns=rename_04_col, inplace=True)
# k04.loc[k04.typr == 12, 'typr'] = 13
# k04.loc[k04.typr == 11, 'typr'] = 12

# k04

In [25]:
k04

,rok,nrg,mn,los,doch,wyd,dgr,wtuk,wp,klm,typr,woj
0,2004,100010111,1466,3.0,2873.00,1957.26,0.0,1952.26,5.00,1,12,2
1,2004,100010121,1497,2.0,1676.88,1675.64,0.0,1675.64,0.00,1,1,2
2,2004,100020111,1466,3.0,3200.00,1151.51,0.0,1149.52,1.99,1,12,2
3,2004,100020112,1497,2.0,976.59,817.71,0.0,817.71,0.00,1,12,2
4,2004,100020121,1497,2.0,1973.94,1951.89,0.0,1951.89,0.00,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
32209,2004,220941221,1587,3.0,1109.00,1043.72,24.0,1043.72,0.00,6,2,32
32210,2004,220951211,2739,1.0,712.85,1239.98,24.5,1239.98,0.00,6,11,32
32211,2004,220951221,1456,4.0,2143.50,1378.65,57.5,1378.65,0.00,6,3,32
32212,2004,220961211,1587,3.0,1272.50,1258.62,29.5,1258.62,0.00,6,6,32


In [23]:
k04.typr.value_counts()

12    6166
1     5816
11    5008
3     4361
2     4122
8     2896
4     1473
6     1055
5      628
9      385
10     218
7       86
Name: typr, dtype: int64

In [4]:
# Load files

# all_kat_paths = glob.glob('BBGD/20*/*kate*.csv')
k06 = pd.read_csv('BBGD\\2006\\br2006_kategorie.csv')
k08 = pd.read_csv('BBGD\\2008\\br2008_kategorie.csv')
k10 = pd.read_csv('BBGD\\2010\\br2010_kategorie.csv')
k12 = pd.read_csv('BBGD\\2012\\br2012_kategorie.csv')
k14 = pd.read_csv('BBGD\\2014\\br2014_kategorie.csv')
k16 = pd.read_csv('BBGD\\2016\\br2016_kategorie.csv')
k18 = pd.read_csv('BBGD\\2018\\br2018_kategorie.csv')

kats = [k04, k06,k08,k10,k12,k14,k16,k18]

# Systemic differences
to_drop = ['region', 'makroreg', 'mn_br04']
to_rename = {'nrg_lp': 'nrg' }
to_order = ['panel', 'gz', 'mn2011']

for i in range(len(kats)):
    kats[i].columns = [x.lower() for x in kats[i].columns]
#     aggregating type category
#     kats[i].loc[kats[i].typr.isin([8,9,10,11]), 'typr'] = 14
    for d in to_drop:
        if d in kats[i].columns:
            kats[i] = kats[i].drop(d, axis=1)
    for r in to_rename:
        if r in kats[i].columns:
            kats[i].rename(columns=to_rename, inplace=True)
    for o in to_order:
        if o in kats[i].columns:
            order = list(kats[i].columns)
            order.remove(o)
            kats[i] = kats[i][order+[o]]

# Unique differences in years
zut_update = {'g_zut': 'g_zut_18', 'd_zut': 'd_zut_18'}
k18.rename(columns=zut_update, inplace=True)

# Compare columns names and number of unique values
var_in_k = []
    
for k in kats:
    var_in_k.append(k.columns)
    types = []
    for col in k:
        v = k[col].unique()
        if len(v) < 20:
            types.append(str(len(v)))
        else:
            types.append('not fixed')
    var_in_k.append(types)
    
pd.DataFrame(var_in_k)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,rok,nrg,mn,los,doch,wyd,dgr,wtuk,wp,klm,typr,woj,None,None,None,None,None,None,None,None,None,None,None,None
1,1,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,6,12,16,None,None,None,None,None,None,None,None,None,None,None,None
2,rok,woj,nrg,mc,mn,g_zut,d_zut,klm,grs,pgrs,typr,los,losz,pgdn,rgdn,dgr,doch,ddp,wyd,wtuk,wp,None,None,None
3,1,16,not fixed,12,not fixed,12,13,6,5,7,13,15,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,None,None,None
4,rok,woj,nrg,mc,mn,g_zut,d_zut,klm,grs,pgrs,typr,los,losz,pgdn,rgdn,dgr,doch,ddp,wyd,wtuk,wp,panel,None,None
5,1,16,not fixed,12,not fixed,12,13,6,5,7,13,15,16,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,2,None,None
6,rok,woj,nrg,mc,mn,g_zut,d_zut,klm,grs,pgrs,typr,los,losz,pgdn,rgdn,dgr,doch,ddp,wyd,wtuk,wp,panel,gz,None
7,1,16,not fixed,12,not fixed,12,13,6,5,7,13,13,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,2,3,None
8,rok,woj,nrg,mc,mn,g_zut,d_zut,klm,grs,pgrs,typr,los,losz,pgdn,rgdn,dgr,doch,ddp,wyd,wtuk,wp,panel,gz,mn2011
9,1,16,not fixed,12,not fixed,12,13,6,5,7,13,14,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,not fixed,2,3,not fixed


In [5]:
# Conversion data base for kategore - all yeras

Convbase_kat = pd.read_excel('200711 Wybór zmiennych.xlsx','klucze do kategorii')

# Index start & end of variable
csi = Convbase_kat[Convbase_kat.code.notna()].index.values
csi = np.append(csi, len(Convbase_kat))
csi = list(zip(csi[:-1]+1,csi[1:]-1))

# Creating dictionaries from values and descriptions
dictionaries=[]
for i ,j in csi:
    keys = Convbase_kat.loc[i:j, 'value']
    values =  Convbase_kat.loc[i:j, 'description']
    convdict = dict(zip(keys, values))
    dictionaries.append(convdict)
    
# Joining dictionaries with labels
conv_kat = pd.DataFrame(Convbase_kat[['code', 'short_name', 'full_name']].dropna().set_index('code'))
conv_kat['dict']=dictionaries

# Codes and short names dictionary
kats_name_dict = dict(zip(conv_kat.index, conv_kat.short_name))

show_wide(conv_kat)

,short_name,full_name,dict
code,,,
woj,województwo,WOJ\nWojewództwo,"{2.0: 'dolnośląskie', 4.0: 'kujawsko-pomorskie', 6.0: 'lubelskie', 8.0: 'lubuskie', 10.0: 'łódzkie', 12.0: 'małopolskie', 14.0: 'mazowieckie', 16.0: 'opolskie', 18.0: 'podkarpackie', 20.0: 'podlas..."
klm,klasa_miejscowości,KLM\nKlasa miejscowości zamieszkania,"{1.0: '500 tys. mieszkańców i więcej', 2.0: '200 - 499 tys. mieszkańców', 3.0: '100 - 199 tys. mieszkańców', 4.0: '20 - 99 tys. mieszkańców', 5.0: 'poniżej 20 tys. mieszkańców', 6.0: 'wieś'}"
gz,stopień_urb,GZ Stopień urbanizacji kraju,"{1.0: 'obszar gęsto zaludniony', 2.0: 'obszar średnio zaludniony', 3.0: 'obszar słabo zaludniony'}"
typr,struktura_gd,TYPR\nTyp biologiczny gospodarstwa domowego,"{1.0: 'małżeństwo bez dzieci', 2.0: 'małżeństwo z 1 dzieckiem na utrzymaniu', 3.0: 'małżeństwo z 2 dzieci na utrzymaniu', 4.0: 'małżeństwo z 3 dzieci na utrzymaniu', 5.0: 'małżeństwo z 4 ( i więce..."
grs,grupa_spo_eko,GRS\nGrupy społeczno-ekonomiczne,"{1.0: 'pracowników', 2.0: 'rolników', 3.0: 'pracujących na własny rachunek', 4.0: 'emerytów i rencistów', 5.0: 'utrzymujących się z niezarobkowych źródeł'}"
pgrs,podgrupa_spo_eko,PGRS \nPodgrupy społeczno-ekonomiczne,"{1.0: 'pracowników na stanowiskach robotniczych', 2.0: 'pracowników na stanowiskach nierobotniczych', 3.0: 'emerytów', 4.0: 'rencistów', 5.0: 'utrzymujących się ze świadczeń społecznych', 6.0: 'ut..."
g_zut_18,głów_źródło_utrz,"D2G, D2D\nGłówne i dodatkowe źródło utrzymania","{1.0: 'praca najemna na stanowisku robotniczym ', 2.0: 'praca najemna na stanowisku nierobotniczym', 3.0: 'użytkowanie gospodarstwa rolnego', 4.0: 'praca na własny rachunek poza gospodarstwem roln..."
d_zut_18,dodat_źródło_utrz,"D2G, D2D\nGłówne i dodatkowe źródło utrzymania","{1.0: 'praca najemna na stanowisku robotniczym ', 2.0: 'praca najemna na stanowisku nierobotniczym', 3.0: 'użytkowanie gospodarstwa rolnego', 4.0: 'praca na własny rachunek poza gospodarstwem roln..."
g_zut,głów_źródło_utrz,"D2G, D2D\nGłówne i dodatkowe źródło utrzymania","{1.0: 'praca najemna na stanowisku robotniczym ', 2.0: 'praca najemna na stanowisku nierobotniczym', 3.0: 'użytkowanie gospodarstwa rolnego', 4.0: 'praca na własny rachunek poza gospodarstwem roln..."


In [6]:
conv_kat.loc['typr','dict']

{1.0: 'małżeństwo bez dzieci',
 2.0: 'małżeństwo z 1 dzieckiem na utrzymaniu',
 3.0: 'małżeństwo z 2 dzieci na utrzymaniu',
 4.0: 'małżeństwo z 3 dzieci na utrzymaniu',
 5.0: 'małżeństwo z 4 ( i więcej) dzieci na utrzymaniu',
 6.0: 'matka z dziećmi na utrzymaniu',
 7.0: 'ojciec z dziećmi na utrzymaniu',
 8.0: 'małżeństwo z przynajmniej 1 dzieckiem na utrzymaniu i innymi osobami',
 9.0: 'matka z dziećmi na utrzymaniu i innymi osobami',
 10.0: 'ojciec z dziećmi na utrzymaniu i innymi osobami',
 11.0: 'inne osoby z dziećmi na utrzymaniu',
 12.0: 'gospodarstwa jednoosobowe',
 13.0: 'pozostałe'}

In [7]:
# Replacing values, renaming columns, changing varibales to categories

kat_var_to_cat = set(['województwo', 'klasa_miejscowości', 'stopień_urb', 'struktura_gd','grupa_spo_eko', 'podgrupa_spo_eko',
                  'głów_źródło_utrz', 'dodat_źródło_utrz', 'okres'])

for i in range(len(kats)):
    k = kats[i]
    for col in k:
        if col in conv_kat.index:
            k[col].replace(conv_kat.loc[col, 'dict'], inplace=True)
    k.rename(columns=kats_name_dict, inplace=True)
    for col in kat_var_to_cat:
        if col in k.columns:
            k[col] = k[col].astype('category')

In [8]:
# Overview of variables
all_var = set()
for i in range(len(kats)):
    all_var.update( set(kats[i].columns))
all_var

collector = []
for i in range(len(kats)):
    k = kats[i]
    ks_col =[]
    for col in all_var:
        if col in k.columns:
            ks_col.append('X')
        else:
            ks_col.append(' ')
    collector.append(ks_col)

overview = pd.DataFrame(collector, columns = all_var, index = range(2004, 2019, 2)).transpose()

overview

,2004,2006,2008,2010,2012,2014,2016,2018
grupa_spo_eko,,X,X,X,X,X,X,X
dodat_źródło_utrz,,X,X,X,X,X,X,X
województwo,X,X,X,X,X,X,X,X
rok,X,X,X,X,X,X,X,X
mn2011,,,,,X,,,
waga,X,X,X,X,X,X,X,X
wydatki_ogółem,X,X,X,X,X,X,X,X
stopień_urb,,,,X,X,X,X,X
głów_źródło_utrz,,X,X,X,X,X,X,X
okres,,,X,X,X,X,X,X


# 01a_1 - all yearrs

In [9]:
# ats[0].d7_04.value_counts()

In [10]:

a04 = pd.read_csv('BBGD\\2004\\br2004_01a_1.csv')
a06 = pd.read_csv('BBGD\\2006\\br2006_01a_1.csv')
a08 = pd.read_csv('BBGD\\2008\\br2008_01a_1.csv')
a10 = pd.read_csv('BBGD\\2010\\br2010_01a_1.csv')
a12 = pd.read_csv('BBGD\\2012\\br2012_01a_1.csv')
a14 = pd.read_csv('BBGD\\2014\\br2014_01a_1.csv')
a16 = pd.read_csv('BBGD\\2016\\br2016_01a_1.csv')
# joining 1b to 2018 data
a18 = pd.read_csv('BBGD\\2018\\br2018_01a_1.csv')
b18 = pd.read_csv('BBGD\\2018\\br2018_01b.csv')
b18.drop(columns=['rok','woj', 'mc'], inplace=True)
a18 = a18.merge(b18, how='left', left_on='NRG_LP', right_on='NRG_LP')

ats = [a04,a06,a08,a10,a12,a14,a16,a18]

# # Systemic differences
# to_drop = ['rok']
# to_rename = {'nrg_lp': 'nrg' }

# getting all variables in data

for i in range(len(ats)):
    year = range(2004, 2019,2)[i]
    ats[i]['rok'] = year
    ats[i].columns = [x.lower()+'_'+str(year)[-2:] for x in ats[i].columns]
#     for d in to_drop:
#         if d in kats[i].columns:
#             kats[i] = kats[i].drop(d, axis=1)
#     for r in to_rename:
#         if r in kats[i].columns:
#             kats[i].rename(columns=to_rename, inplace=True)

# O - variable missing
# X - infinite value variable
# N - number of dscrete values
all_var = set()
for i in range(len(ats)):
    all_var.update(set(ats[i].columns))
all_var = set([v[:-3] for v in all_var])

collector = []
for i in range(len(ats)):
    a = ats[i]
    as_col =[]
    for col in all_var:
        trunk_col = [v[:-3] for v in a.columns]
        if col in trunk_col:
            index = trunk_col.index(col)
            unique = a[a.columns[index]].unique()
            nr_u = len(unique)
            if nr_u < 20:
                as_col.append(nr_u)
            else:
                as_col.append('X')
        else:
            as_col.append('O')
    collector.append(as_col)
overview = pd.DataFrame(collector, columns = all_var, index = range(2004, 2019, 2))

overview

,d5_20_3,d5_17,d6_5_b,d5_8_3,d623,d2_d,d5_13_3,d6_3_5,d5_2,d5_35_3,d621,d5_37_2i,d5_22,r11_4,d5_15_6,d6_3_4,d5_41,d5_36_2i,d625,d6_5_5,d6_2440,d6_5_h,d6_35,d5_50m,nrg_lp,d5_44_3,d343,d5_15_1,d5_37_1i,d7_25,r11_6,d6_1,d5_26_2,d5_22_1,d7_24,d5_4_1,d7_1,d6_10,d6_14,d6_29,d7_23,d6_5_1,d5_23_2,d5_37_1,klm,d5_17_2,d6_2_2,d5_18_1,d5_24_10,d6_4,d221,d6_30,r11_5,d6_482,d6_5_6,d6_5_3,d5_24_11,d5_14,d5_22_6,d6_7,d5_23_3,d5_24_09,d5_15_3,d5_19,d346,d5_39,d6_15,d5_11,d5_45_2,d6_3_1,d6_222,d6_473,r11_1,d5_20_4,d5_8,d6_154,d5_16,d5_25,d6_241,d6_16,d5_20_6,d5_48,d5_4_2,d5_24_08,d5_15_5,d6_282,d2g,gse,d22,d5_3,d5_15_7,d5_45_3,mc,d5_22_7,d6_13,d5_44,d5_49m,d5_46_1,d5_22_2,d5_8_1,d6_12,d4c,d5_24_06,d5_8_2,d5_49,d6_221,d3_2_2,d5_50l,d5_49l,d5_19_1,d5_24_12,d5_13_1,d6_5_a,d6_5_g,d5_17_1,d3_12,d5_15_12,d5_18,d5_19_2,d5_10,d63,d5_13,okr,d6_5_7,d5_46_2,d39,d5_38_2,nrod,d5_15_11,d5_15_8,d6_19,d347,poza,d6_41,d6_2_5,d341,d6_31,d5_5,d6_244,d61,d6_21,d5_30,d3_11,d6_5_f,d6_20,d5_45m,d344,d2d,d3_4,d5_28,sos,d5_43,d3_21,d5_22_4,d5_44_1,d6_231,r11_3,d5_24,d6_8,d5_45l,d6_281,d3_13,d5_27_1,d5_45_1,d6_18,d6_5_d,d5_20_8,d7,trb,d6_3,d6_152,d3_1,d3_2,d5_15_4,d31,d6_3_3,d6_3_2,d5_21,d6_232,d5_38_1,d2_g,d5_19_4,d5_27_2,d5_7,d5_23,d7_22,d6_5_2,d222,d6_283,d6_472,d35,d5_18_2,d5_15_10,d21,d38,d23,nrg,los,d5_24_02,d5_20_1,d5_26,d6_242,d6_2_1,d6_5_4,d24,d622,d5_20_2,d5_8_4,d25,d6_153,d5_19_3,d5_22_5,d5_37_2,d5_38_3,d3_3,d5_20,d345,gse6,woj,d5_24_03,d5_35_1,d37,r8,d6_5,d32,zut,d6_243,d6_6,d5_6,d5_20_7,rok,d36,d6_481,d7_3,d5_1,d6_2_3,d6_471,d5_26_1,d6_33,d6_17,d33,d624,r11_2,d5_24_04,d5_27,d6_2_4,d5_13_2,d5_15_2,d5_4,d6_2,d5_9,d5_38,d342,d5_32,d7_21,d5_36_1i,d6_5_c,d5_20_5,d6_11,d6_2220,d6_5_e,d6_151,spoza,d5_15_9,d5_44_2,d5_24_07,d5_22_3,d5_15,d5_24_05,d5_24_01,d3_2_1,d348,d3_22,d5_12,d5_35_2,r12,d6_5_8,d6_9,d5_23_1
2004,O,O,O,O,X,O,O,O,O,O,X,O,O,O,O,O,O,O,X,O,O,O,O,O,O,O,2,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,6,O,O,O,O,O,X,O,O,O,O,O,O,O,O,O,O,O,O,O,2,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,3,X,O,O,O,O,O,O,O,O,O,O,O,O,10,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,6,O,O,O,O,3,O,O,O,O,O,5,X,O,O,2,O,O,O,5,O,O,O,O,O,O,3,O,O,O,X,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,3,12,O,O,O,O,O,11,O,O,O,O,O,O,O,O,O,O,O,O,X,O,O,X,O,O,2,14,3,X,O,O,O,O,O,O,O,X,X,O,O,11,O,O,O,O,O,O,O,3,3,16,O,O,16,O,O,4,7,O,O,O,O,1,X,O,O,O,O,O,O,O,O,6,X,O,O,O,O,O,O,O,O,O,O,2,O,O,O,O,O,O,O,O,O,X,O,O,O,O,O,O,O,O,2,O,O,O,O,O,O,O
2006,O,O,O,O,O,13,O,O,O,O,O,O,O,O,O,O,O,O,O,O,2,O,5,O,O,O,O,O,O,X,O,4,O,O,X,O,5,3,12,2,X,O,O,O,O,O,O,O,O,X,O,2,O,2,O,O,O,O,O,X,O,O,O,O,O,O,O,O,O,O,O,2,O,O,O,2,O,O,2,2,O,O,O,O,O,2,O,O,O,O,O,O,12,O,X,O,O,O,O,O,X,O,O,O,O,3,O,O,O,O,O,O,O,O,O,2,O,O,O,O,O,O,3,O,O,O,O,10,O,O,3,O,O,3,O,O,3,O,O,O,2,O,2,O,2,O,O,O,X,O,O,O,X,O,O,X,O,O,6,O,2,2,O,O,3,O,O,O,O,X,2,O,X,O,O,O,O,O,X,O,12,O,O,O,O,X,O,O,2,2,O,O,O,O,O,O,X,O,O,O,O,2,O,O,O,O,O,O,O,2,O,O,O,O,3,O,O,O,16,O,O,O,O,2,O,O,2,6,O,O,1,O,5,6,O,O,2,O,3,2,O,O,O,O,O,O,O,O,O,X,O,O,O,O,X,O,O,O,3,3,O,2,O,O,O,O,O,O,O,O,O,O,X,O,O,O,O,7,O
2008,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,5,O,O,O,O,O,O,X,O,4,O,O,X,O,5,3,12,2,X,O,O,O,O,O,O,O,O,X,O,2,O,3,O,O,O,O,O,X,O,O,O,O,O,O,4,O,O,O,3,2,O,O,O,O,O,O,2,2,O,O,O,O,O,2,12,O,O,O,O,O,12,O,X,O,O,O,O,O,X,O,O,O,O,3,O,O,O,O,O,O,O,O,O,2,O,O,O,O,O,O,3,O,O,O,O,10,O,O,3,O,O,3,O,O,3,O,2,O,2,O,2,O,2,O,O,13,X,O,O,O,X,O,O,X,O,O,6,O,2,2,O,O,3,O,O,O,O,X,O,O,X,O,O,O,O,O,X,O,O,O,O,O,O,X,O,O,2,2,O,O,O,O,O,O,X,O,O,O,O,2,O,O,O,O,O,O,O,O,O,O,O,O,3,O,O,O,16,O,O,O,O,2,O,O,2,6,O,O,1,O,3,6,O,O,2,O,3,2,O,O,O,O,O,O,O,O,O,X,O,O,O,O,X,O,O,O,3,O,O,O,O,O,O,O,O,O,O,O,O,O,X,O,O,O,O,8,O
2010,O,O,O,X,O,O,O,O,X,O,O,O,O,O,2,O,O,O,O,O,O,O,O,O,O,O,O,2,O,O,O,5,O,O,O,O,O,O,O,O,O,O,2,O,O,3,X,X,O,O,O,O,O,O,O,O,O,3,O,O,2,O,2,O,O,O,O,X,2,O,O,O,O,O,O,O,2,2,O,O,O,O,O,O,2,O,12,O,O,19,2,2,12,O,O,O,O,3,O,X,O,O,O,X,O,O,O,O,O,2,O,O,O,O,3,2,O,O,2,X,O,4,3,O,3,O,O,8,O,2,O,O,O,O,X,O,O,6,O,O,O,3,2,O,O,O,O,13,X,3,O,O,X,O,O,O,O,O,O,O,O,2,O,2,O,O,O,O,O,6,O,O,X,2,O,O,O,O,O,O,O,2,O,8,O,O,O,O,O,O,O,X,2,O,O,O,X,O,O,O,3,O,X,O,O,O,O,X,O,O,2,O,O,O,3,O,O,O,16,O,O,O,O,O,O,O,O,O,6,O,1,O,O,O,4,X,O,O,O,O,

In [11]:

# Conversion data base for kategore - all yeras
Convbase_at = pd.read_excel('200711 Wybór zmiennych.xlsx','klucze do 01a')

# Index start & end of variable
csi = Convbase_at[Convbase_at.short_name.notna()].index.values
csi = np.append(csi, len(Convbase_at))
csi = list(zip(csi[:-1]+1,csi[1:]-1))

# Creating dictionaries from values and descriptions
dictionaries=[]
for i ,j in csi:
    keys = Convbase_at.loc[i:j, 'value']
    values =  Convbase_at.loc[i:j, 'description']
    convdict = dict(zip(keys, values))
    dictionaries.append(convdict)
    
# Joining dictionaries with labels
conv_at = pd.DataFrame(Convbase_at[list(range(2004, 2019,2))+['short_name', 'full_name']])
conv_at = conv_at.loc[conv_at.short_name.notna()]
conv_at['dict']=dictionaries
conv_at.reset_index(drop=True, inplace=True)

# Year labels sufix to variables
for col in range(2004, 2019,2):
    conv_at[col] = conv_at[col].apply(lambda x: (str(x)+'_'+str(col)[-2:])  if not pd.isnull(x) else np.nan)
    
# Dictionary for renaming values   
zz = conv_at[list(range(2004, 2019,2))].values
nam_keys=[]
nam_values=[]   
for z in range(len(zz)):
    var = zz[z]
    for label in var:
        if pd.notnull(label):
            nam_keys.append(label)
            nam_values.append(conv_at.loc[z, 'short_name'])       
ats_name_dict = dict(zip(nam_keys,nam_values))

show_wide(conv_at)

,2004,2006,2008,2010,2012,2014,2016,2018,short_name,full_name,dict
0,d32_04,d6_1_06,d6_1_08,d5_1_10,d5_1_12,d5_1_14,d5_1_16,r8_18,rodzaj_budynku,rodzaj budynku,"{1.0: 'budynek wielorodzinny', 2.0: 'dom jednorodzinny w zabudowie szeregowej (również bliźniak)', 3.0: 'dom jednorodzinny wolnostojący', 4.0: 'inny'}"
1,d33_04,d6_6_06,NaN,NaN,NaN,NaN,NaN,NaN,rok_budowy,Uproszczony kres wybudowania budynku,"{1.0: '1800, 1946', 2.0: '1946, 1960', 3.0: '1961, 1980', 4.0: '1981, 1995', 5.0: '1996, 2006', 6.0: '1996, 2006'}"
2,NaN,NaN,d6_6_08,d5_5_10,d5_5_12,d5_5_14,d5_5_16,d5_1_18,rok_budowy,Uproszczony kres wybudowania budynku,"{1.0: '1800, 1946', 2.0: '1946, 1960', 3.0: '1961, 1980', 4.0: '1981, 1995', 5.0: '1996, 2006', 6.0: '2007, 2011', 7.0: '2011, 2018'}"
3,NaN,d6_8_06,d6_8_08,d5_6_10,d5_6_12,d5_6_14,d5_6_16,d5_2_18,własność,Własność budynku,"{1.0: 'osoby fizycznej', 2.0: 'spółdzielni mieszkaniowej (mieszkania spółdzielcze, własnościowe i lokatorskie)', 3.0: 'gminy, Skarbu Państwa, zakładu pracy', 4.0: 'towarzystwa budownictwa społeczn..."
4,d31_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nieobciążona', 3.0: 'spółdzielcze obciążone', 4.0: 'spółdzielcze nieobciążone', 5.0: 'spółdzielcze nieobciążone', 6.0: 'najem cena poniżej rynkowej', 7.0..."
5,NaN,d6_9_06,NaN,NaN,NaN,NaN,NaN,NaN,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nieobciążona', 3.0: 'spółdzielcze nieobciążone', 4.0: 'najem cena rynkowa', 5.0: 'najem cena poniżej rynkowej', 6.0: 'najem bez opłat', 7.0: 'inny'}"
6,NaN,NaN,d6_9_08,d5_7_10,NaN,NaN,NaN,NaN,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nieobciążona', 3.0: 'spółdzielcze obciążone', 4.0: 'spółdzielcze nieobciążone', 5.0: 'najem cena rynkowa', 6.0: 'najem cena poniżej rynkowej', 7.0: 'naje..."
7,NaN,NaN,NaN,NaN,d5_7_12,d5_7_14,d5_7_16,d5_3_18,tytuł_do_mieszkania,Tytuł prawny do mieszkania,"{1.0: 'własność obciążona', 2.0: 'własność nieobciążona', 3.0: 'spółdzielcze obciążone', 4.0: 'spółdzielcze nieobciążone', 5.0: 'spółdzielcze nieobciążone', 6.0: 'najem cena rynkowa', 7.0: 'najem ..."
8,d344_04,d6_33_06,d6_33_08,d5_30_10,d5_30_12,d5_30_14,d5_21_16,d5_17_18,sieć_ciepłownicza,Czy mieszkanie wyposażone jest w ciepłą wodę?,"{1.0: 'tak, z sieci', 2.0: 'tak, ogrzewaną lokalnie', 3.0: 'nie'}"
9,d345_04,d6_41_06,d6_41_08,d5_38_10,d5_38_12,d5_38_14,d5_23_16,d5_19_18,sieć_gazowa,Czy mieszkanie wyposażone jest w gaz?,"{1.0: 'tak, z sieci', 2.0: 'tak, z butli', 3.0: 'nie'}"


In [12]:
# Replacing values, renaming columns, changing varibales to categories

at_var_to_cat = set(['rodzaj_budynku', 'rok_budowy', 'własność', 'tytuł_do_mieszkania', 'tytuł_do_mieszkania',
                 'tytuł_do_mieszkania', 'sieć_ciepłownicza', 'sieć_gazowa', 'okres'])
lisy = list(range(2004, 2019,2))
for i in range(len(ats)):
    a = ats[i]
    for col in a.columns:
        if conv_at[lisy[i]].isin([col]).any():
            index = Index(conv_at[lisy[i]]).get_loc(col)
            dictionary = conv_at.loc[index, 'dict']
            a[col].replace(dictionary, inplace=True) # this is ok 
    a.rename(columns=ats_name_dict, inplace=True)
    for cc in at_var_to_cat:
        if cc in a.columns:
            a[cc] = a[cc].astype('category')

In [13]:
# Overview of variables
selec_var = set(conv_at.short_name.tolist())
collector = []
for i in range(len(ats)):
    a = ats[i]
    as_col =[]
    for col in selec_var:
        if col in a.columns:
            as_col.append('X')
        else:
            as_col.append(' ')
    collector.append(as_col)
overview = pd.DataFrame(collector, columns = selec_var, index = range(2004, 2019, 2)).transpose()
overview

,2004,2006,2008,2010,2012,2014,2016,2018
rodzaj_budynku,X,X,X,X,X,X,X,X
liczba_pięter,,X,X,X,X,X,,
powierzchnia_użytkowa,X,X,X,X,X,X,X,X
gd_id,X,X,X,X,X,X,X,X
sieć_gazowa,X,X,X,X,X,X,X,X
tytuł_do_mieszkania,X,X,X,X,X,X,X,X
okres,X,X,,,,,,
piętro,,X,X,X,X,X,,
rok_budowy,X,X,X,X,X,X,X,X
własność,,X,X,X,X,X,X,X


# Combining the data sets

In [14]:
kiv = kats_important_variables = pd.Index(['gd_id','rok', 'województwo', 'nrg', 'waga', 'głów_źródło_utrz','klasa_miejscowości', 'grupa_spo_eko', 'struktura_gd',
                            'liczba_osób', 'ilczba_osób_żyw','przychody_netto','dochód_rozporządzany', 'wydatki_ogółem', 'okres', 'stopień_urb'])

aiv = ats_important_variables = pd.Index(['sieć_ciepłownicza', 'rok_budowy', 'sieć_gazowa', 'gd_id', 'liczba_pięter', 'okres', 'piętro', 'pokoje',
                           'powierzchnia_użytkowa','powierzchnia_zajm', 'rodzaj_budynku', 'tytuł_do_mieszkania', 'własność'])
bbgd= []
for i in range(len(kats)):
    ikiv = kats[i].columns.intersection(kiv)
    iaiv = ats[i].columns.intersection(aiv)
    bbgd.append(kats[i][ikiv].merge(ats[i][iaiv], how='left', left_on='gd_id', right_on='gd_id'))
bbgd[0].head(5)

,rok,gd_id,waga,liczba_osób,dochód_rozporządzany,wydatki_ogółem,klasa_miejscowości,struktura_gd,województwo,tytuł_do_mieszkania,rodzaj_budynku,rok_budowy,sieć_ciepłownicza,sieć_gazowa,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,okres
0,2004,100010111,1466,3.0,2873.00,1957.26,500 tys. mieszkańców i więcej,pozostałe,dolnośląskie,własność nieobciążona,dom jednorodzinny w zabudowie szeregowej (równ...,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",70,70,5,dobrane
1,2004,100010121,1497,2.0,1676.88,1675.64,500 tys. mieszkańców i więcej,małżeństwo bez dzieci,dolnośląskie,własność nieobciążona,dom jednorodzinny w zabudowie szeregowej (równ...,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",70,70,5,pierwszy
2,2004,100020111,1466,3.0,3200.00,1151.51,500 tys. mieszkańców i więcej,pozostałe,dolnośląskie,własność nieobciążona,dom jednorodzinny wolnostojący,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",98,49,3,dobrane
3,2004,100020112,1497,2.0,976.59,817.71,500 tys. mieszkańców i więcej,pozostałe,dolnośląskie,własność nieobciążona,dom jednorodzinny wolnostojący,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",98,49,3,dobrane
4,2004,100020121,1497,2.0,1973.94,1951.89,500 tys. mieszkańców i więcej,małżeństwo bez dzieci,dolnośląskie,własność nieobciążona,dom jednorodzinny wolnostojący,"1961, 1980","tak, ogrzewaną lokalnie","tak, z sieci",200,200,8,pierwszy


# Saving preprocessed data

In [15]:
for i in range(len(bbgd)):
    lisy[i]
    bbgd[i].to_csv('BBGD/preprocessed_data/BBGD_'+str(lisy[i])+'.csv', index=False)

# Loading preprocessed data

In [16]:
lisy = list(range(2004, 2019,2))
bb_var_to_cat = pd.Index(['sieć_ciepłownicza', 'rok_budowy', 'dodat_źródło_utrz', 'sieć_gazowa', 'grupa_spo_eko', 'głów_źródło_utrz',
                                'klasa_miejscowości', 'okres', 'podgrupa_spo_eko', 'rodzaj_budynku', 'stopień_urb', 'struktura_gd',
                                'tytuł_do_mieszkania', 'województwo', 'własność'])
bbgd=[]
for i in lisy:
    bb = pd.read_csv('BBGD/preprocessed_data/BBGD_'+str(i)+'.csv')
    ibb_var_to_cat = bb.columns.intersection(bb_var_to_cat)
    bb[ibb_var_to_cat] = bb[ibb_var_to_cat].astype('category')
    bbgd.append(bb)

In [17]:
for i in bbgd:
    display(i.head(5))

,rok,gd_id,waga,liczba_osób,dochód_rozporządzany,wydatki_ogółem,klasa_miejscowości,struktura_gd,województwo,tytuł_do_mieszkania,rodzaj_budynku,rok_budowy,sieć_ciepłownicza,sieć_gazowa,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,okres
0,2004,100010111,1466,3.0,2873.00,1957.26,500 tys. mieszkańców i więcej,pozostałe,dolnośląskie,własność nieobciążona,dom jednorodzinny w zabudowie szeregowej (równ...,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",70,70,5,dobrane
1,2004,100010121,1497,2.0,1676.88,1675.64,500 tys. mieszkańców i więcej,małżeństwo bez dzieci,dolnośląskie,własność nieobciążona,dom jednorodzinny w zabudowie szeregowej (równ...,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",70,70,5,pierwszy
2,2004,100020111,1466,3.0,3200.00,1151.51,500 tys. mieszkańców i więcej,pozostałe,dolnośląskie,własność nieobciążona,dom jednorodzinny wolnostojący,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",98,49,3,dobrane
3,2004,100020112,1497,2.0,976.59,817.71,500 tys. mieszkańców i więcej,pozostałe,dolnośląskie,własność nieobciążona,dom jednorodzinny wolnostojący,"1800, 1946","tak, ogrzewaną lokalnie","tak, z sieci",98,49,3,dobrane
4,2004,100020121,1497,2.0,1973.94,1951.89,500 tys. mieszkańców i więcej,małżeństwo bez dzieci,dolnośląskie,własność nieobciążona,dom jednorodzinny wolnostojący,"1961, 1980","tak, ogrzewaną lokalnie","tak, z sieci",200,200,8,pierwszy


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,rodzaj_budynku,liczba_pięter,piętro,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa,okres
0,2006,dolnośląskie,100010111,2528,emerytura,500 tys. mieszkańców i więcej,emerytów i rencistów,gospodarstwa jednoosobowe,1,1.00,1726.68,1526.68,1007.40,budynek wielorodzinny,11,3,"1981, 1995",spółdzielni mieszkaniowej (mieszkania spółdzie...,spółdzielcze nieobciążone,37,37,2,"tak, z sieci","tak, z sieci",pierwszy
1,2006,dolnośląskie,100010121,1533,emerytura,500 tys. mieszkańców i więcej,emerytów i rencistów,małżeństwo z 1 dzieckiem na utrzymaniu,3,3.00,1682.44,1682.44,1379.81,budynek wielorodzinny,10,5,"1981, 1995","gminy, Skarbu Państwa, zakładu pracy",najem bez opłat,62,62,3,"tak, z sieci","tak, z sieci",pierwszy
2,2006,dolnośląskie,100010211,1472,praca najemna na stanowisku robotniczym,500 tys. mieszkańców i więcej,pracowników,inne osoby z dziećmi na utrzymaniu,5,5.07,3936.00,3436.00,2223.38,budynek wielorodzinny,10,0,"1981, 1995",spółdzielni mieszkaniowej (mieszkania spółdzie...,spółdzielcze nieobciążone,48,48,2,"tak, z sieci","tak, z sieci",pierwszy
3,2006,dolnośląskie,100010221,1472,emerytura,500 tys. mieszkańców i więcej,emerytów i rencistów,matka z dziećmi na utrzymaniu i innymi osobami,5,5.00,4387.40,4387.40,3486.85,budynek wielorodzinny,10,4,"1981, 1995",spółdzielni mieszkaniowej (mieszkania spółdzie...,własność nieobciążona,54,54,3,"tak, z sieci","tak, z sieci",pierwszy
4,2006,dolnośląskie,100010311,1472,praca najemna na stanowisku robotniczym,500 tys. mieszkańców i więcej,pracowników,małżeństwo z 3 dzieci na utrzymaniu,5,5.00,2427.00,2327.00,1591.06,budynek wielorodzinny,10,8,"1981, 1995","gminy, Skarbu Państwa, zakładu pracy",najem bez opłat,62,62,3,"tak, z sieci","tak, z sieci",pierwszy


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,okres,rodzaj_budynku,liczba_pięter,piętro,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa
0,2008,dolnośląskie,100010111,2133,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,gospodarstwa jednoosobowe,1,1,1800.00,1300.00,1531.44,drugi,budynek wielorodzinny,4,1,"1981, 1995",spółdzielni mieszkaniowej (mieszkania spółdzie...,spółdzielcze obciążone,53,53,2,"tak, z sieci","tak, z sieci"
1,2008,dolnośląskie,100010121,1768,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,małżeństwo z 2 dzieci na utrzymaniu,4,4,4195.00,3895.00,2416.43,drugi,budynek wielorodzinny,4,3,"1981, 1995",spółdzielni mieszkaniowej (mieszkania spółdzie...,spółdzielcze obciążone,59,59,3,"tak, z sieci","tak, z sieci"
2,2008,dolnośląskie,100010211,2133,praca na własny rachunek poza gospodarstwem ro...,500 tys. mieszkańców i więcej,pracujących na własny rachunek,gospodarstwa jednoosobowe,1,1,1800.00,1800.00,1305.78,drugi,budynek wielorodzinny,4,2,"1961, 1980",spółdzielni mieszkaniowej (mieszkania spółdzie...,najem cena rynkowa,68,40,1,"tak, ogrzewaną lokalnie","tak, z sieci"
3,2008,dolnośląskie,100010221,1991,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,inne osoby z dziećmi na utrzymaniu,6,6,4764.33,3764.33,2779.69,drugi,budynek wielorodzinny,2,2,"1800, 1946","gminy, Skarbu Państwa, zakładu pracy",najem bez opłat,72,72,3,"tak, ogrzewaną lokalnie","tak, z sieci"
4,2008,dolnośląskie,100010311,2133,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,gospodarstwa jednoosobowe,1,1,2250.00,250.00,1193.63,drugi,budynek wielorodzinny,3,3,"1800, 1946",osoby fizycznej,własność obciążona,51,51,3,"tak, ogrzewaną lokalnie","tak, z sieci"


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,okres,stopień_urb,rodzaj_budynku,liczba_pięter,piętro,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa
0,2010,zachodniopomorskie,100010111,1638,praca najemna na stanowisku robotniczym,200 - 499 tys. mieszkańców,pracowników,małżeństwo z 1 dzieckiem na utrzymaniu,3,3.00,2500.00,2200.00,2476.57,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,3,"1800, 1946","gminy, Skarbu Państwa, zakładu pracy",najem bez opłat,87,87,3,"tak, ogrzewaną lokalnie","tak, z sieci"
1,2010,zachodniopomorskie,100010121,1249,praca na własny rachunek poza gospodarstwem ro...,200 - 499 tys. mieszkańców,pracujących na własny rachunek,małżeństwo bez dzieci,2,1.71,8200.00,8200.00,3609.66,drugi,obszar gęsto zaludniony,budynek wielorodzinny,3,2,"1800, 1946",osoby fizycznej,własność nieobciążona,56,56,3,"tak, ogrzewaną lokalnie","tak, z sieci"
2,2010,zachodniopomorskie,100010211,1249,emerytura,200 - 499 tys. mieszkańców,emerytów i rencistów,małżeństwo bez dzieci,2,2.00,2300.00,2100.00,2003.09,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,4,"1961, 1980",osoby fizycznej,własność nieobciążona,48,48,2,"tak, z sieci","tak, z sieci"
3,2010,zachodniopomorskie,100010221,1249,emerytura,200 - 499 tys. mieszkańców,emerytów i rencistów,pozostałe,2,0.89,1901.87,1701.87,1300.76,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,4,"1800, 1946","gminy, Skarbu Państwa, zakładu pracy",najem bez opłat,35,35,2,"tak, ogrzewaną lokalnie","tak, z sieci"
4,2010,zachodniopomorskie,100010311,1249,praca najemna na stanowisku nierobotniczym,200 - 499 tys. mieszkańców,pracowników,matka z dziećmi na utrzymaniu,2,2.26,4360.33,4060.33,3383.53,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,2,"1981, 1995",osoby fizycznej,własność nieobciążona,64,64,3,"tak, z sieci","tak, z sieci"


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,okres,stopień_urb,rodzaj_budynku,liczba_pięter,piętro,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa
0,2012,dolnośląskie,101010111,1968,emerytura,500 tys. mieszkańców i więcej,emerytów i rencistów,gospodarstwa jednoosobowe,1,0.87,3220.0,3220.0,2091.07,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,2,"1996, 2006",osoby fizycznej,własność nieobciążona,87,87,3,"tak, z sieci","tak, z sieci"
1,2012,dolnośląskie,101010121,1833,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,małżeństwo z przynajmniej 1 dzieckiem na utrzy...,4,3.10,10190.0,10190.0,6686.40,drugi,obszar gęsto zaludniony,budynek wielorodzinny,3,2,"1800, 1946",osoby fizycznej,własność nieobciążona,76,76,3,"tak, ogrzewaną lokalnie","tak, z sieci"
2,2012,dolnośląskie,101010211,1968,emerytura,500 tys. mieszkańców i więcej,emerytów i rencistów,gospodarstwa jednoosobowe,1,0.50,1283.5,1263.5,1259.80,drugi,obszar gęsto zaludniony,budynek wielorodzinny,2,1,"1800, 1946",osoby fizycznej,własność nieobciążona,78,78,2,"tak, ogrzewaną lokalnie","tak, z sieci"
3,2012,dolnośląskie,101010221,1194,emerytura,500 tys. mieszkańców i więcej,emerytów i rencistów,małżeństwo bez dzieci,2,2.07,3318.5,3318.5,1393.51,drugi,obszar gęsto zaludniony,budynek wielorodzinny,3,1,"1961, 1980",osoby fizycznej,spółdzielcze nieobciążone,40,40,2,"tak, z sieci","tak, z sieci"
4,2012,dolnośląskie,101010311,1833,praca najemna na stanowisku robotniczym,500 tys. mieszkańców i więcej,pracowników,pozostałe,4,4.00,4400.0,4000.0,1125.36,drugi,obszar gęsto zaludniony,dom jednorodzinny w zabudowie szeregowej (równ...,0,0,"1961, 1980",osoby fizycznej,własność nieobciążona,100,100,3,"tak, ogrzewaną lokalnie","tak, z sieci"


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,okres,stopień_urb,rodzaj_budynku,liczba_pięter,piętro,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa
0,2014,świętokrzyskie,1,1412,emerytura,200 - 499 tys. mieszkańców,emerytów i rencistów,małżeństwo bez dzieci,2,1.97,3710.00,3560.00,2839.15,drugi,obszar gęsto zaludniony,budynek wielorodzinny,9,2,"1981, 1995",osoby fizycznej,własność nieobciążona,57,57,3,"tak, ogrzewaną lokalnie","tak, z sieci"
1,2014,świętokrzyskie,2,1827,emerytura,200 - 499 tys. mieszkańców,emerytów i rencistów,gospodarstwa jednoosobowe,1,1.00,2250.00,2000.00,1159.00,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,0,"1961, 1980",osoby fizycznej,własność nieobciążona,60,60,3,"tak, ogrzewaną lokalnie","tak, z sieci"
2,2014,świętokrzyskie,3,1412,praca najemna na stanowisku robotniczym,200 - 499 tys. mieszkańców,pracowników,małżeństwo bez dzieci,2,2.00,4850.00,4500.00,3128.53,drugi,obszar gęsto zaludniony,budynek wielorodzinny,9,3,"1961, 1980",osoby fizycznej,własność nieobciążona,42,42,2,"tak, ogrzewaną lokalnie","tak, z sieci"
3,2014,świętokrzyskie,4,1827,emerytura,200 - 499 tys. mieszkańców,emerytów i rencistów,gospodarstwa jednoosobowe,1,0.96,2888.63,2538.63,2471.93,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,0,"1961, 1980",osoby fizycznej,własność nieobciążona,47,47,2,"tak, ogrzewaną lokalnie","tak, z sieci"
4,2014,świętokrzyskie,5,1412,praca najemna na stanowisku robotniczym,200 - 499 tys. mieszkańców,pracowników,małżeństwo bez dzieci,2,2.00,3600.00,3400.00,2035.25,drugi,obszar gęsto zaludniony,budynek wielorodzinny,4,3,"1961, 1980",osoby fizycznej,własność nieobciążona,43,43,2,"tak, ogrzewaną lokalnie","tak, z sieci"


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,okres,stopień_urb,rodzaj_budynku,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa
0,2016,dolnośląskie,1,1053,praca najemna na stanowisku robotniczym,wieś,pracowników,pozostałe,4,4.00,3949.0,3739.0,3088.39,drugi,obszar gęsto zaludniony,dom jednorodzinny wolnostojący,"1800, 1946",osoby fizycznej,własność nieobciążona,100,100,4,"tak, ogrzewaną lokalnie","tak, z butli"
1,2016,dolnośląskie,2,989,praca najemna na stanowisku robotniczym,wieś,pracowników,pozostałe,3,2.71,4567.6,2767.6,2955.57,drugi,obszar gęsto zaludniony,dom jednorodzinny wolnostojący,"1981, 1995",osoby fizycznej,własność nieobciążona,200,200,6,"tak, ogrzewaną lokalnie","tak, z butli"
2,2016,dolnośląskie,3,989,praca najemna na stanowisku robotniczym,wieś,pracowników,małżeństwo z 1 dzieckiem na utrzymaniu,3,3.00,2400.0,1600.0,2785.11,drugi,obszar gęsto zaludniony,budynek wielorodzinny,"1961, 1980",osoby fizycznej,własność nieobciążona,76,76,3,"tak, ogrzewaną lokalnie","tak, z butli"
3,2016,dolnośląskie,4,989,praca najemna na stanowisku robotniczym,wieś,pracowników,małżeństwo z 1 dzieckiem na utrzymaniu,3,3.00,3388.9,3278.9,2297.92,drugi,obszar gęsto zaludniony,dom jednorodzinny wolnostojący,"1800, 1946",osoby fizycznej,własność nieobciążona,120,120,5,"tak, ogrzewaną lokalnie","tak, z butli"
4,2016,dolnośląskie,5,2011,emerytura,poniżej 20 tys. mieszkańców,emerytów i rencistów,gospodarstwa jednoosobowe,1,1.03,1600.0,1200.0,1038.17,drugi,obszar słabo zaludniony,budynek wielorodzinny,"1800, 1946",osoby fizycznej,własność nieobciążona,48,48,2,nie,"tak, z sieci"


,rok,województwo,gd_id,waga,głów_źródło_utrz,klasa_miejscowości,grupa_spo_eko,struktura_gd,liczba_osób,ilczba_osób_żyw,przychody_netto,dochód_rozporządzany,wydatki_ogółem,okres,stopień_urb,rok_budowy,własność,tytuł_do_mieszkania,powierzchnia_użytkowa,powierzchnia_zajm,pokoje,sieć_ciepłownicza,sieć_gazowa,rodzaj_budynku
0,2018,dolnośląskie,1,1294.89,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,małżeństwo bez dzieci,2,1.74,14120.00,7120.0,3231.23,drugi,obszar gęsto zaludniony,"2007, 2011",osoby fizycznej,własność obciążona,42,42,2,"tak, z sieci",nie,budynek wielorodzinny
1,2018,dolnośląskie,2,1294.89,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,małżeństwo bez dzieci,2,1.87,5510.00,5150.0,3051.89,drugi,obszar gęsto zaludniony,"1961, 1980",osoby fizycznej,najem bez opłat,34,34,2,"tak, z sieci","tak, z sieci",budynek wielorodzinny
2,2018,dolnośląskie,3,1975.37,praca najemna na stanowisku nierobotniczym,500 tys. mieszkańców i więcej,pracowników,małżeństwo z 1 dzieckiem na utrzymaniu,3,2.74,8203.99,7900.0,16667.82,drugi,obszar gęsto zaludniony,"1961, 1980",osoby fizycznej,własność obciążona,54,54,3,"tak, z sieci","tak, z sieci",budynek wielorodzinny
3,2018,dolnośląskie,4,1677.42,wynajem nieruchomości,500 tys. mieszkańców i więcej,emerytów i rencistów,gospodarstwa jednoosobowe,1,0.77,1920.00,1820.0,1579.66,drugi,obszar gęsto zaludniony,"1961, 1980",osoby fizycznej,własność nieobciążona,62,62,3,"tak, ogrzewaną lokalnie","tak, z sieci",dom jednorodzinny w zabudowie szeregowej (równ...
4,2018,dolnośląskie,5,1294.89,wynajem nieruchomości,500 tys. mieszkańców i więcej,emerytów i rencistów,małżeństwo bez dzieci,2,2.00,3500.00,3300.0,1536.59,drugi,obszar gęsto zaludniony,"1961, 1980",osoby fizycznej,własność nieobciążona,63,63,3,"tak, z sieci","tak, z sieci",budynek wielorodzinny


In [18]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////     !!!FUCK YEAH!!!    //////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

SyntaxError: invalid syntax (<ipython-input-18-8ec9e7abb732>, line 1)